In [1]:
import pandas as pd           # data mnipulation
import numpy as np            # number manipulation/crunching
import matplotlib.pyplot as plt  # plotting
from sklearn.metrics import  accuracy_score, classification_report 
import openpyxl
# Train Test split
from sklearn.model_selection import train_test_split
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from anchor import utils
from anchor import anchor_tabular
from sklearn.model_selection import LeaveOneOut
import lime
import lime.lime_tabular
from collections import Counter
import sys
import os

hepatitis = pd.read_csv("hepatitis.csv")
hepatitis['ALB'] = hepatitis['ALB'].replace(to_replace=np.nan,value = hepatitis['ALB'].median())
hepatitis['ALP'] = hepatitis['ALP'].replace(to_replace=np.nan,value = hepatitis['ALP'].median())
hepatitis['ALT'] = hepatitis['ALT'].replace(to_replace=np.nan,value = hepatitis['ALT'].median())
hepatitis['AST'] = hepatitis['AST'].replace(to_replace=np.nan,value = hepatitis['AST'].median())
hepatitis['BIL'] = hepatitis['BIL'].replace(to_replace=np.nan,value = hepatitis['BIL'].median())
hepatitis['CHE'] = hepatitis['CHE'].replace(to_replace=np.nan,value = hepatitis['CHE'].median())
hepatitis['CHOL'] = hepatitis['CHOL'].replace(to_replace=np.nan,value = hepatitis['CHOL'].median())
hepatitis['CREA'] = hepatitis['CREA'].replace(to_replace=np.nan,value = hepatitis['CREA'].median())
hepatitis['GGT'] = hepatitis['GGT'].replace(to_replace=np.nan,value = hepatitis['GGT'].median())
hepatitis['PROT'] = hepatitis['PROT'].replace(to_replace=np.nan,value = hepatitis['PROT'].median())

hepatitis['Outcome'].value_counts()
Y = hepatitis['Outcome']
target_names=["0", "1"]
    

X = hepatitis[['Age',	'Sex',	'ALB',	'ALP',	'ALT',	'AST',	'BIL',	'CHE',	'CHOL',	'CREA',	'GGT',	'PROT']]
maximums = np.amax(np.array(X), axis=0)
#X = diabetes.iloc[:,:-1].values
X_featurenames = X.columns


# Split the data into train and test data:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
counter = Counter(Y)
scale_pos_weight = counter[0] / counter[1]
model = xgb.XGBClassifier(scale_pos_weight = scale_pos_weight)
model.fit(X_train, Y_train)
pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, pred)
print("Accuracy:", accuracy)
print(classification_report(Y_test, pred, target_names=["0", "1"]))

dataframe_columns_Anchors = []
test_index = "test_index"
dataframe_columns_Anchors.append(test_index)
for feature in X_featurenames:
    feature_lower_interval = feature + "_lower_interval"
    feature_upper_interval = feature + "_upper_interval"
    feature_rank = feature + "_rank"
    dataframe_columns_Anchors.append(feature_lower_interval)
    dataframe_columns_Anchors.append(feature_upper_interval)
    dataframe_columns_Anchors.append(feature_rank)
accuracy_classifier = "accuracy_classifier"
precision = "precision"
coverage = "coverage"
prediction = "prediction"
true_class = "true_class"
dataframe_columns_Anchors.append(accuracy_classifier)
dataframe_columns_Anchors.append(precision)
dataframe_columns_Anchors.append(coverage)
dataframe_columns_Anchors.append(prediction)
dataframe_columns_Anchors.append(true_class)
df_Anchors = pd.DataFrame(columns = dataframe_columns_Anchors)

dataframe_columns_LIME = []
test_index = "test_index"
dataframe_columns_LIME.append(test_index)
for feature in X_featurenames:
    feature_lower_interval = feature + "_lower_interval"
    feature_upper_interval = feature + "_upper_interval"
    feature_probability = feature + "_probability"
    feature_sign = feature + "_sign"
    feature_rank = feature + "_rank"
    dataframe_columns_LIME.append(feature_lower_interval)
    dataframe_columns_LIME.append(feature_upper_interval)
    dataframe_columns_LIME.append(feature_probability)
    dataframe_columns_LIME.append(feature_sign)
    dataframe_columns_LIME.append(feature_rank)
intercept = "intercept"
local_pred = "local_pred"
probability_no_hepatitis = "probability_no_hepatitis"
probability_hepatitis = "probability_hepatitis"
accuracy_classifier = "accuracy_classifier"
true_class = "true_class"
dataframe_columns_LIME.append(intercept)
dataframe_columns_LIME.append(local_pred)
dataframe_columns_LIME.append(probability_no_hepatitis)
dataframe_columns_LIME.append(probability_hepatitis)
dataframe_columns_LIME.append(accuracy_classifier)
dataframe_columns_LIME.append(true_class)
df_LIME = pd.DataFrame(columns = dataframe_columns_LIME)

cv = LeaveOneOut()

number_iterations = 100
for train_ix, test_ix in cv.split(X):
    print(test_ix[0])
    for count in range(number_iterations):
        model = xgb.XGBClassifier(scale_pos_weight = scale_pos_weight)
        X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
        Y_train, Y_test = Y[train_ix], Y[test_ix]
        model.fit(X_train, Y_train)  
        explainer = anchor_tabular.AnchorTabularExplainer(class_names=target_names,
                                                        feature_names=X_featurenames,train_data=X_train)
        exp = explainer.explain_instance(X_test[0], model.predict, threshold=0.99)
        list = exp.names()
        row_Anchors = []
        row_Anchors.append(test_ix[0])
        for feature in X_featurenames:
            ok = 0
            for j in range(len(list)):
                if list[j].find(feature) != -1 :
                    ok = 1
                    #print(list[j])
                    rank = j + 1
                    if list[j].count('.') == 2:
                        lower_interval = list[j][0 : list[j].find('<')]
                        upper_interval = list[j][list[j].find('=') + 2 : len(list[j]) + 1]
                    elif list[j].count('.') == 1 and list[j].find('<') != -1:
                        lower_interval = 0
                        if(list[j].find('=') != -1):
                            upper_interval = list[j][list[j].find('=') + 2 : len(list[j]) + 1]
                        else:
                            upper_interval = list[j][list[j].find('<') + 2 : len(list[j]) + 1]
                    elif list[j].count('.') == 1 and list[j].find('>') != -1:
                        upper_interval = maximums[X_featurenames.to_list().index(feature)]
                        if(list[j].find('=') != -1):
                            lower_interval = list[j][list[j].find('=') + 2 : len(list[j]) + 1]
                        else:
                            lower_interval = list[j][list[j].find('>') + 2 : len(list[j]) + 1]
            if ok == 0:
                lower_interval = -1
                upper_interval = -1
                rank = -1
            #print(lower_interval)
            #print(upper_interval)
            #print(rank)
            row_Anchors.append(lower_interval)
            row_Anchors.append(upper_interval)
            row_Anchors.append(rank)
        pred = model.predict(X_test)
        accuracy_classifier = accuracy_score(Y_test, pred)
        precision = exp.precision()
        coverage = exp.coverage()
        prediction =  explainer.class_names[model.predict(X_test[0].reshape(1, -1))[0]]
        true_class = target_names[Y_test[test_ix[0]]]
        row_Anchors.append(accuracy_classifier)
        row_Anchors.append(precision)
        row_Anchors.append(coverage)
        row_Anchors.append(prediction)
        row_Anchors.append(true_class)
        #print(row)
        #print(df)
        df_Anchors.loc[test_ix[0] * number_iterations + count] = row_Anchors
        #print(df)
        #-----------------------------------------------------------------
        old_stdout = sys.stdout # backup current stdout
        sys.stdout = open(os.devnull, "w")
        explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names = X_featurenames, class_names = ['0', '1'], feature_selection = "lasso_path", discretize_continuous = True, discretizer = "quartile", verbose = True, mode = 'classification')
        exp = explainer.explain_instance(X_test[0], model.predict_proba)
        #exp.show_in_notebook(show_table = True, show_all = True)
        sys.stdout = old_stdout
        list = exp.as_list()
        intercept = exp.intercept[1]
        local_pred = exp.local_pred[0]
        row_LIME = []
        row_LIME.append(test_ix[0])
        for feature in X_featurenames:
            upper_interval = -1
            lower_interval = -1
            probability = -1
            sign = -1
            rank = -1
            for j in range(len(X_featurenames)):
                if j < len(list) and list[j] is not None and list[j][0].find(feature) != -1 :
                    probability = list[j][1]
                    rank = j + 1
                    if probability > 0 :
                        sign = "positive"
                    else :
                        sign = "negative"
                    if list[j][0].count('.') == 2:
                        lower_interval = list[j][0][0 : list[j][0].find('<')]
                        upper_interval = list[j][0][list[j][0].find('=') + 2 : len(list[j][0]) + 1]
                    elif list[j][0].count('.') == 1 and list[j][0].find('<') != -1:
                        lower_interval = 0
                        if(list[j][0].find('=') != -1):
                            upper_interval = list[j][0][list[j][0].find('=') + 2 : len(list[j][0]) + 1]
                        else:
                            upper_interval = list[j][0][list[j][0].find('<') + 2 : len(list[j][0]) + 1]
                    elif list[j][0].count('.') == 1 and list[j][0].find('>') != -1:
                        upper_interval = maximums[X_featurenames.to_list().index(feature)]
                        if(list[j][0].find('=') != -1):
                            lower_interval = list[j][0][list[j][0].find('=') + 2 : len(list[j][0]) + 1]
                        else:
                            lower_interval = list[j][0][list[j][0].find('>') + 2 : len(list[j][0]) + 1]
                    #print(lower_interval)
                    #print(upper_interval)
            row_LIME.append(lower_interval)
            row_LIME.append(upper_interval)
            row_LIME.append(probability)
            row_LIME.append(sign)
            row_LIME.append(rank)
        probability_no_hepatitis = model.predict_proba([X_test[0]])[0,0]
        probability_hepatitis = model.predict_proba([X_test[0]])[0,1]
        pred = model.predict(X_test)
        accuracy_classifier = accuracy_score(Y_test, pred)
        true_class = target_names[Y_test[test_ix[0]]]
        row_LIME.append(intercept)
        row_LIME.append(local_pred)
        row_LIME.append(probability_no_hepatitis)
        row_LIME.append(probability_hepatitis)
        row_LIME.append(accuracy_classifier)
        row_LIME.append(true_class)
        #print(row_LIME)
        df_LIME.loc[test_ix[0] * number_iterations + count] = row_LIME

df_Anchors.to_excel("outputHepatitisAnchorLOOCV.xlsx")
df_LIME.to_excel("outputHepatitisLIMELOOCV.xlsx")

# print('Anchor: %s' % (' AND '.join(exp.names())))
# print('Precision: %.2f' % exp.precision())
# print('Coverage: %.2f' % exp.coverage())

Accuracy: 0.9512195121951219
              precision    recall  f1-score   support

           0       0.99      0.96      0.97       113
           1       0.64      0.90      0.75        10

    accuracy                           0.95       123
   macro avg       0.82      0.93      0.86       123
weighted avg       0.96      0.95      0.95       123

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
18